## Generate static graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "prepilot"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
import scaffolding 
from plots import *

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_ei
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_re = defaultdict(lambda: 'Other',dic_re)

In [ ]:
dic_pur

In [ ]:
tq = scaffolding.get_time_query(year, month)

In [ ]:
participant_ct_df = scaffolding.load_all_participant_trips(program, tq)

In [ ]:
labeled_ct = scaffolding.filter_labeled_trips(participant_ct_df)

In [ ]:
expanded_ct = scaffolding.expand_userinputs(labeled_ct)

In [ ]:
expanded_ct = scaffolding.data_quality_check(expanded_ct)
expanded_ct.shape

In [ ]:
## Mapping new labels with dictionaries
expanded_ct['Trip_purpose']= expanded_ct['purpose_confirm'].map(dic_pur)
expanded_ct['Mode_confirm']= expanded_ct['mode_confirm'].map(dic_re)
expanded_ct['Replaced_mode']= expanded_ct['replaced_mode'].map(dic_re)

In [ ]:
# Energy Impact Calculation
expanded_ct = scaffolding.energy_cal(expanded_ct,dic_ei)

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)
quality_text = scaffolding.get_quality_text(participant_ct_df, expanded_ct)

# Distance vs. Energy_Impact (kWH) by Mode_confirm

In [ ]:
data=expanded_ct.loc[(expanded_ct['distance_miles'] <= 40)].sort_values(by=['Energy_Impact(kWH)'], ascending=False)   
x='Energy_Impact(kWH)'
y='distance_miles'
legend ='Mode_confirm'
plot_title="Sketch of Energy Impact (kWH) by Travel Mode Selected\n%s" % quality_text
file_name ='sketch_distance_energy_impact%s.png' % file_suffix
distancevsenergy(data,x,y,legend,plot_title,file_name)

# Energy_Impact (kWH)

In [ ]:
#eirp : energy impact replaced_mode
eirc=expanded_ct.groupby('Replaced_mode').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
eirc.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
eirc = eirc.reset_index()
eirc = eirc.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)
eirc['boolean'] = eirc['Sketch of Total Energy_Impact(kWH)'] > 0

#eimc : energy impact mode_confirm
eimc=expanded_ct.groupby('Mode_confirm').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
eimc.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
eimc = eimc.reset_index()
eimc = eimc.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)


subset1 = eirc [['Replaced_mode', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset1.rename(columns = {'Replaced_mode':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Replaced_Mode' }, inplace=True)

subset2 = eimc [['Mode_confirm', 'Sketch of Total Energy_Impact(kWH)']].copy()
subset2.rename(columns = {'Mode_confirm':'Transport Mode','Sketch of Total Energy_Impact(kWH)':'Mode_Confirm' }, inplace=True)

df_plot = pd.merge(subset1, subset2, on="Transport Mode")
df = pd.melt(df_plot , id_vars=['Transport Mode'], value_vars=['Replaced_Mode','Mode_Confirm'], var_name='selection')
df.rename(columns = {'value':'Energy Impact (kWH)'}, inplace = True)

In [ ]:
df

In [ ]:
df= df.sort_values(by=['Energy Impact (kWH)'], ascending=False)
x= 'Energy Impact (kWH)'
y= 'Transport Mode'
color = 'selection'
plot_title="Sketch of Energy Impact (kWH) by Transport Mode\n%s" % quality_text
file_name ='sketch_all_energy_impact%s.png' % file_suffix
overeall_energy_impact(x,y,color,df,plot_title,file_name)

# Sketch of Total Energy_Impact (kWH) by Replaced_mode

In [ ]:
net_energy_saved = round(sum(eirc['Sketch of Total Energy_Impact(kWH)']), 2)

x = eirc['Sketch of Total Energy_Impact(kWH)']
y = eirc['Replaced_mode']
color =eirc['boolean']

plot_title="Sketch of Energy Impact for all confirmed trips \n Contribution by mode towards a total of %s (kWH) \n%s" % (net_energy_saved, quality_text)
file_name ='sketch_all_mode_energy_impact%s.png' % file_suffix
energy_impact(x,y,color,plot_title,file_name)

# Sketch of Energy Impact by E-bike trips

In [ ]:
data_eb = expanded_ct.query("Mode_confirm == 'Pilot ebike'")
# ebei : ebike energy impact
ebei=data_eb.groupby('Replaced_mode').agg({'Energy_Impact(kWH)': ['sum', 'mean']},)
ebei.columns = ['Sketch of Total Energy_Impact(kWH)', 'Sketch of Average Energy_Impact(kWH)']
ebei= ebei.reset_index()
ebei = ebei.sort_values(by=['Sketch of Total Energy_Impact(kWH)'], ascending=False)
ebei['boolean'] = ebei['Sketch of Total Energy_Impact(kWH)'] > 0
net_energy_saved = round(sum(ebei['Sketch of Total Energy_Impact(kWH)']), 2)

x = ebei['Sketch of Total Energy_Impact(kWH)']
y = ebei['Replaced_mode']
color =ebei['boolean']

plot_title="Sketch of Energy Impact of E-Bike trips\n Contribution by replaced mode towards a total of %s (kWH)\n %s" % (net_energy_saved, quality_text)
file_name ='sketch_energy_impact_ebike%s.png' % file_suffix
energy_impact(x,y,color,plot_title,file_name)

# CO2 Emissions Calculation

In [ ]:
expanded_ct = scaffolding.CO2_emissions(expanded_ct,'Replaced_mode', 'distance_miles')
expanded_ct = scaffolding.CO2_emissions(expanded_ct,'Mode_confirm', 'distance_miles')
expanded_ct['CO2_impact'] = expanded_ct['Replaced_mode_CO2'] - expanded_ct['Mode_confirm_CO2']

# Sketch of Total CO2 Emissions by Replaced_mode

In [ ]:
co2rp=expanded_ct.groupby('Replaced_mode').agg({'CO2_impact': ['sum', 'mean']},)
co2rp.columns = ['total_lb_CO2_emissions', 'average_lb_CO2_emission']
co2rp = co2rp.reset_index()
co2rp= co2rp.sort_values(by=['total_lb_CO2_emissions'], ascending=False)
co2rp['boolean'] = co2rp['total_lb_CO2_emissions'] > 0


net_CO2_saved = round(sum(co2rp['total_lb_CO2_emissions']), 2)

x = co2rp['total_lb_CO2_emissions']
y = co2rp['Replaced_mode']
color =co2rp['boolean']

plot_title="Sketch of CO2 Emissions Impact for all confirmed trips \n Contribution by mode towards a total of %s (kWH) \n%s" % (net_CO2_saved, quality_text)
file_name ='sketch_all_mode_CO2impact%s.png' % file_suffix
CO2_impact(x,y,color,plot_title,file_name)

## Sketch of  CO2 Emissions Impact by E-bike trips

In [ ]:
data_eb = expanded_ct.query("Mode_confirm == 'Pilot ebike'")

ebco2=data_eb.groupby('Replaced_mode').agg({'CO2_impact': ['sum', 'mean']},)
ebco2.columns = ['total_lb_CO2_emissions', 'average_lb_CO2_emission']
ebco2 = ebco2.reset_index()
ebco2 = ebco2.sort_values(by=['total_lb_CO2_emissions'], ascending=False)
ebco2['boolean'] = ebco2['total_lb_CO2_emissions'] > 0
net_CO2_emissions = round(sum(ebco2['total_lb_CO2_emissions']), 2)

x = ebco2['total_lb_CO2_emissions']
y = ebco2['Replaced_mode']
color = ebco2['boolean']

plot_title="Sketch of Total Pounds of CO2 Emissions of E-Bike trips\n Contribution by replaced mode towards a total of %s (lb CO2 Emissions )\n %s" % (net_CO2_emissions, quality_text)
file_name ='sketch_CO2impact_ebike%s.png' % file_suffix
CO2_impact(x,y,color,plot_title,file_name)